# **Insper - Eletromagnetismo e Ondulatória** #


**Laboratório 2 - Indutância e Transformador**



In [ ]:
import numpy as np
from numpy import linalg
from cmath import *
import math
import matplotlib.pyplot as plt

In [ ]:
Capacitores = [150*1e-9, 0.1*1e-6, 0.47*1e-6, 1e-6, 4.7*1e-6]
Css = ['150nF', '0.1uF', '0.47uF', '1uF', '4.7uF']
fs = np.arange(1, 100*1e3, 10)
Rc = np.arange(5,30,5)
k = 0.2
Uf = 12



#frequência angular
#w = 2*math.pi*f


#dispersão

#indutância mútua
#M = 0.96 * math.sqrt(L1*L2)


#reatâncias indutivas
#XL1 = 1j*w*L1
#XL2 = 1j*w*L2
#XM = 1j*w*M

In [ ]:
#def CalcularTransformador(Uf, Rc):
    #Z=np.array([[R1+XL1, -XM],[-XM, XL2+R2+Rc]])
    #V=np.array([Uf,0])
    #i=np.dot(linalg.inv(Z),V)
    #return i[0], i[1]


resultados = []

def CalcularTransformadorSerie(Uf, c, f, Rc):

    C1 = c
    C2 = c
    w = 2*np.pi*f
    L1 = 1/(w**2*C1)
    L2 = 1/(w**2*C2)
    R1 = 2
    R2 = 2
    R1 = R1 + (R1*f)/(1e5)
    R2 = R2 + (R2*f)/(1e5)

    M = k * sqrt(L1 * L2)
    XL1 = complex(0, w * L1)
    XL2 = complex(0, w * L2)
    XC1 = complex(0, -1/(w * C1))
    XC2 = complex(0, -1/(w * C2))
    XM = complex(0, w * M)
    
    for rc in Rc:
        Z = np.array([
            [R1 + XL1 + XC1, -XM],
            [-XM, R2 + XL2 + rc]
        ])
        V = np.array([Uf, 0])
        i = np.dot(linalg.inv(Z), V)

        resultados.append((i[0], i[1]))

    return resultados 

def CalcularTransformadorParalelo(Uf, c, f):
    C1 = c
    C2 = c
    w = 2*np.pi*f
    L1 = 1/(w**2*C1)
    L2 = 1/(w**2*C2)
    R1 = 2
    R2 = 2
    R1 = R1 + (R1*f)/(1e5)
    R2 = R2 + (R2*f)/(1e5)

    M = k * sqrt(L1 * L2)
    XL1 = complex(0, w * L1)
    XL2 = complex(0, w * L2)
    XC1 = complex(0, -1/(w * C1))
    XC2 = complex(0, -1/(w * C2))
    XM = complex(0, w * M)

    for rc in Rc:
        Zeq_C2_RC = (rc * XC2) / (rc + XC2)

        Z = np.array([
            [R1 + XL1 + XC1, -XM],
            [-XM, R2 + XL2 + Zeq_C2_RC]
        ])
        V = np.array([Uf, 0])
        i = np.dot(linalg.inv(Z), V)
        resultados.append((i[0], i[1], Zeq_C2_RC))

    return resultados 


d) Agora calcule as correntes $i_1$ e $i_2$ para as duas condições de carga. Assim que calcular as correntes, calcule também $V_2$.

IMPORTANTE: para calcular as relações $N_1/N_2$, $V_1/V_2$ e $i_2/i_1$, você precisará usar o módulo dos fasores. Para calcular o módulo de um número complexo, use "np.abs(número complexo)".

In [ ]:
#Rcarga = 10

#i1, i2 = CalcularTransformador(Ufonte, Rcarga)

#print('Com resistor na saída de %.2f Ω:' %Rcarga)
#print('i1 (eficaz) = %.6f A' %np.abs(i1))
#print('i2 (eficaz) = %.6f A' %np.abs(i2))
#print('V1 (eficaz): %.6f V' %Ufonte)
#print('V2 (eficaz): %.6f V' %(np.abs(i2*Rcarga)))

#print('Relação N1/N2: %.3f' %(np.abs(sqrt(L1/L2))))
#print('Relação i2/i1: %.6f' %(np.abs(i2)/np.abs(i1)))
#print('Relação V1/V2: %.6f' %(np.abs(Ufonte)/np.abs(i2*Rcarga)))



Com resistor na saída de 10.00 Ω:
i1 (eficaz) = 0.175843 A
i2 (eficaz) = 0.506050 A
V1 (eficaz): 30.120000 V
V2 (eficaz): 5.060503 V
Relação N1/N2: 3.003
Relação i2/i1: 2.877847
Relação V1/V2: 5.951977
